# Lesson4 ニューラル翻訳モデルを作ってみよう（Seq2Seq, Attention）

## 目次

- Section1 Checkクイズの解答
- Section2 実装①
    - 2.0 データの用意
    - 2.1 モデル構築
    - 2.2 モデルの学習
    - 2.3 モデルによる予測
    - 2.4 モデルの可視化
    - 2.5 機械翻訳の評価について（補足）

## Section1 Checkクイズの解答

問題1: 3, 問題2: 1, 問題3; 2, 問題4: 3

## Section2 実装①

LSTMを使ったSeq2Seqモデルで英日機械翻訳を行ってみましょう。

使用するデータセット、train.enとtrain.jaの中身は次のようになっています.

train.enの中身 (英語の文)
```
i can 't tell who will arrive first .
many animals have been destroyed by men .
i 'm in the tennis club .
︙
```

train.jaの中身(日本語の文, 対訳)
```
誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
私 は テニス 部員 で す 。
︙
```
(データセットにはTanaka Corpus ( http://www.edrdg.org/wiki/index.php/Tanaka_Corpus )の一部を抽出した 
small_parallel_enja: 50k En/Ja Parallel Corpus for Testing SMT Methods ( https://github.com/odashi/small_parallel_enja ) を使っています.)

### 2.0 データの用意

まずはデータの読み込みです。

読み込む際、文頭を表す仮想単語（**BOS**, Beginning Of Sentence）として`<s>`、文末を表す仮想単語（**EOS**, End Of Sentence）として`<\s>`を付加します。

また、BOS, EOSをつけた文章について、Tokenizerによって数値化を行います。

最後に、バッチ処理のため、各系列の長さをそろえておきます。これには`keras.preprocessing.sequence.pad_sequences`を用います。

詳しくは、https://keras.io/ja/preprocessing/sequence/#pad_sequences を参照してください。

<img src='../figures/preprocess.png' width='80%'>

### 2.1 モデル構築

ここでは、LSTMを使用してSeq2Seqモデルを構築します。

Embeddingレイヤーでは`mask_zero=True`を引数として指定することで、計算上先程のパディング部分を無視するようにしています。

また、Recurrentレイヤーに対する`return_state=True`や`return_sequences=True`の指定をLSTMレイヤーの生成時に行っています。

なお、Functional APIによるモデル構築であることに注意してください。

<img src='../figures/model.png'>

なお、図の各レイヤーは以下のように対応することに注意してください。

1. 符号化器Embeddingレイヤー： EncoderのEmbedding
2. 符号化器再帰レイヤー：　Encoder(LSTM)
3. 復号化器Embeddingレイヤー： DecoderのEmbedding
4. 復号化器再帰レイヤー： Decoder(LSTM)
5. 復号化器出力レイヤー： Dense

#### LSTMレイヤーの補足

LSTMの出力に関して補足の説明をします。以下の図はLSTMの模式図です。

<img src='../figures/lstm.png' width='50%'>

**1. LSTMレイヤーはオプションがない場合はレイヤーの出力を返します。**

```
output = LSTM()(x)
```

なお、このときの出力(output)とはLSTMの最後の隠れ状態$h$のことを指します。

図を3timestepとみる場合は $h_{t+2}$にあたります。

**2.  引数に`return_state=True`を指定されているときは隠れ状態も返します。**

```
output, state_h, state_c = LSTM(return_state=True)(x)
```

このときの`state_h`と`state_c`はそれぞれLSTMの最後の隠れ状態$h$とセル状態$c$となります。

図を3timestepとみる場合は `state_h`は $h_{t+2}$ 、`state_c`は$c_{t+2}$にあたります。

このとき、`output=state_h`となっていることに注意してください。

また、以下のコードを説明すると、

```
_, *encoder_states = LSTM(hid_dim, return_state=True)(encoder_embedded)
```

outputは今回は参照しないので`_`(アンダーバー)を使っており、`*`(スター)を使うことで`encoder_states=[state_h, state_c]`となるような代入を行っています。このような使い方はpython一般の使い方ですので疑問に思った方はpythonの復習をおすすめします。

**3.  引数に`return_sequences=True`を指定されているときは系列も返します。**

```
outputs, state_h, state_c = LSTM(return_state=True, return_sequences=True)(x)
```

このとき`outputs`は系列すべての出力を含みます。

図を3timestepとみる場合は、`outputs`に $h_{t}$、$h_{t+1}$、$h_{t+2}$のすべての出力を含むということになります。

### 2.2 モデルの学習

モデルの学習時には、教師データとして1時点先の単語を示すデータを入力します。(`train_target`)

学習時にはDecoderの入力に教師データを用います。

<img src='../figures/training.png'>

### 2.3 モデルによる生成

先程学習したモデルを使用して、系列を生成してみましょう。

そのためにまずは学習したモデルを組み込んだ、系列生成用のモデルを構築します。

学習時との違いは、復号化器が1ステップずつ実行できるよう、状態ベクトルの入力と出力をモデルの定義に加えている点です。

(また、1ステップ前の状態を引き継いで生成が可能になるように、復号化器のモデルの初期状態を指定可能にしています。)

生成する際のDecoderの入力には翻訳先の教師データは用いません。

<img src='../figures/prediction.png'>

このモデルを使用した生成（予測）を行ってみましょう。

生成では、未知のデータに対してモデルを適用するので正解ラベルはわかりません。

そこで、代わりに前のステップで予測した単語を各ステップでの入力とします。

そして, 系列の終わりを表す単語 (`</s>`) が出力されるまで繰り返します。（最初の入力は`<s>`を使用します）

### 2.4 モデルの可視化

### 2.5 機械翻訳の評価について（補足）

これまで、基本的に学習したモデルの良し悪しについては損失をベースに考えてきました。

ですが機械翻訳の場合、損失と翻訳の精度が必ずしも一致しません。

というのも、翻訳においては、単純に各単語が一致しているか否か以上に、意味的な繋がりや表現の流暢さが重要となるためです。

また、必ずしも語順についても一致している必要はありません。

そこで、そうした翻訳タスク特有の性質を反映した評価指標が必要となります。その代表例として、**BLEUスコア**が挙げられます。

BLEUスコアは、n-gram（連続n単語. 主にn=4）がどれだけ生成文と正解文で共有されているかなどを考慮した指標となっています。

機械翻訳は本講座の主目的ではないので、ここではその詳細な算出方法等は触れませんが、興味がある方はスクリプト（http://www.nltk.org/_modules/nltk/translate/bleu_score.html ）をのぞいてみてください。

このBLEUスコアの他にも、機械翻訳の評価指標がいくつか提案されています。

代表的なものの比較は、

N. Graham, "文レベルの機械翻訳評価尺度に関する調査", 研究報告自然言語処理, vol. 2013-NL-212, no. 7, pp. 1–8, 2013. (http://phontron.com/paper/neubig13nl212.pdf)

などにまとめられているので、機械翻訳に興味のあるかたは参照してみてください。